In [1]:
import keras.backend as K
from keras import losses

from keras.optimizers import Adam
from keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Conv2DTranspose, Dropout, UpSampling2D
from keras.models import Model
from keras import regularizers 
from keras.layers.normalization import BatchNormalization as bn
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint, Callback

from keras.preprocessing.image import ImageDataGenerator


import os
import random
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from matplotlib.pyplot import imshow,title,show





import keras


from keras.utils import plot_model
import pydot
import keras
import pydotplus
from keras.utils.vis_utils import model_to_dot

keras.utils.vis_utils.pydot = pydot

Using TensorFlow backend.


In [2]:
import cv2
from matplotlib import pyplot as plt 

In [19]:
def unet_resnet(input_shape =[256,256,3], dropout_rate=0.2,l2_lambda = 0.0002):
    #Encoder
    #input = Input(shape = input_shape, name = "input")
    model0 = keras.applications.resnet50.ResNet50(include_top=False, weights=None, input_tensor=None, input_shape=input_shape, pooling=None)
    #model0.layers.pop(176)
    #model0.layers.pop(175)
    
    out12 = model0.layers[12]
    out38 = model0.layers[38]
    out102 = model0.layers[102]
    output145 = model0.layers[145]
    
    activation12 = out12.output
    activation38 = out38.output
    activation102 = out102.output
    activation145 = output145.output
    
    
    final_activation_output = model0.layers[174]
    final_activation = final_activation_output.output
    
    #Decoder
    upconv6 = Conv2DTranspose(512,(2, 2), strides=(2, 2), padding='same')(final_activation)
    upconv6 = Dropout(dropout_rate)(upconv6)
    upconv6  = keras.layers.UpSampling2D(size=(0.5,0.5))(upconv6)
    concat6 = concatenate([activation145, upconv6], name = "concat6")
    conv6_1 = Conv2D(256, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv6_1")(concat6)
    conv6_1 = bn(name = "conv6_1_bn")(conv6_1)
    conv6_2 = Conv2D(256, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv6_2")(conv6_1)
    conv6_2 = bn(name = "conv6_2_bn")(conv6_2)

    upconv7 = Conv2DTranspose(1024,(2, 2), strides=(2, 2), padding='same')(conv6_2)
    upconv7 = Dropout(dropout_rate)(upconv7)
    upconv7  = keras.layers.UpSampling2D(size=(1,1))(upconv7)
    concat7 = concatenate([activation102, upconv7], name = "concat7")
    conv7_1 = Conv2D(128, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv7_1")(concat7)
    conv7_1 = bn(name = "conv7_1_bn")(conv7_1)
    conv7_2 = Conv2D(128, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv7_2")(conv7_1)
    conv7_2 = bn(name = "conv7_2_bn")(conv7_2)

    upconv8 = Conv2DTranspose(256,(2, 2), strides=(2, 2), padding='same')(conv7_2)
    upconv8 = Dropout(dropout_rate)(upconv8)
    upconv8  = keras.layers.UpSampling2D(size=(2,2))(upconv8)
    concat8 = concatenate([activation38, upconv8], name = "concat8")
    conv8_1 = Conv2D(64, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv8_1")(concat8)
    conv8_1 = bn(name = "conv8_1_bn")(conv8_1)
    conv8_2 = Conv2D(64, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv8_2")(conv8_1)
    conv8_2 = bn(name = "conv8_2_bn")(conv8_2)

    upconv9 = Conv2DTranspose(64,(2, 2), strides=(2, 2), padding='same')(conv8_2)
    upconv9 = Dropout(dropout_rate)(upconv9)
    upconv9  = keras.layers.UpSampling2D(size=(0.5,0.5))(upconv9)
    concat9 = concatenate([activation12, upconv9], name = "concat9")
    conv9_1 = Conv2D(32, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv9_1")(concat9)
    conv9_1 = bn(name = "conv9_1_bn")(conv9_1)
    conv9_2 = Conv2D(32, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv9_2")(conv9_1)
    conv9_2 = bn(name = "conv9_2_bn")(conv9_2)
    dropout = Dropout(dropout_rate)(conv9_2)
   
    up6 = UpSampling2D(size=(8,8))(concat6)
    up7 = UpSampling2D(size=(4,4))(concat7)
    up8 = UpSampling2D(size=(2,2))(concat8)
    
    up6 = UpSampling2D(size=(4,4))(up6)
    up7 = UpSampling2D(size=(4,4))(up7)
    up8 = keras.layers.UpSampling2D(size=(2,2))(up8)
    dropout = UpSampling2D(size=(4,4))(dropout)
    
    concate1 = concatenate([up6,up7,up8,dropout])

    conv10 = Conv2D(1, (1, 1), padding = "same", activation = 'sigmoid', name = "conv10")(concate1)


    model = Model(model0.input, conv10)
    #model = conv10
    return model
    


In [8]:
#input_shape = [256,256,3]

In [9]:
#plot_model(model,to_file='/home/deeptek4/Desktop/resnet_unet.png',show_shapes=True)

In [10]:
#model1.summary()

In [11]:
#model0.summary()

In [12]:
#len(model0.layers)

In [13]:
#len(model1.layers)

In [7]:
X_train = np.load('/home/deeptek4/MY_WORK/KAGGLE_COMP1/data/Submission2_Data/Unet/unet_train_channel3.npy')
Y_train = np.load('/home/deeptek4/MY_WORK/KAGGLE_COMP1/data/Submission2_Data/Unet/unet_ytrain_for_channel3.npy')
X_val = np.load('/home/deeptek4/MY_WORK/KAGGLE_COMP1/data/Submission2_Data/Unet/unet_val_channel3.npy')
Y_val = np.load('/home/deeptek4/MY_WORK/KAGGLE_COMP1/data/Submission2_Data/Unet/unet_yval_for_channel3.npy')

In [14]:
X_train  = X_train/255
X_val = X_val/255

In [15]:
def dice(true, preds):  
    scores  = []
    for i in range(len(true)):
        im1 = true[i]
        im2 = preds[i]
        intersection = np.logical_and(im1, im2)
        score = 2. * intersection.sum() / (im1.sum() + im2.sum())
        scores.append(score)
    return np.array(scores).mean()#, scores

def dice_score(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f)+K.epsilon())

def dice_loss(y_true, y_pred):
    return 1 - dice_score(y_true, y_pred)



class dice_call(Callback):
    def __init__(self,validation_data):
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]

    def on_epoch_end(self, epoch, logs={}):

        if(epoch%1==0):
            print("Calc DICE")
            y_pred_val = np.around(self.model.predict(self.x_val,verbose=1))
            try:
                dice_val = dice(self.y_val, y_pred_val)
            except:
                pass
            print("DICE VAL {0}".format(dice_val))

        return
    
    
smooth = 1.0
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f1 = K.flatten(K.round(y_pred))
    intersection = K.sum(y_true_f * y_pred_f1)
    return  (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f1) + smooth)

In [17]:
input_size = 256

In [20]:
input_shape = [input_size, input_size, 3]
dropout_rate = 0.3
l2_lambda = 0.0002


K.clear_session()
model = unet_resnet(input_shape, dropout_rate, l2_lambda)
#model1 = u_net(input_shape,dropout_rate,l2_lambda)
adam = Adam(lr = 0.0001)
model.compile(optimizer = adam, loss = 'binary_crossentropy', metrics = [dice_coef])

/home/deeptek4/anaconda3/envs/deeptek3/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
W0812 14:13:49.981523 140108015470400 deprecation.py:506] From /home/deeptek4/anaconda3/envs/deeptek3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0812 14:13:49.999838 140108015470400 deprecation_wrapper.py:119] From /home/deeptek4/anaconda3/envs/deeptek3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:2018: The name tf.image.resize_nearest_neighbor is deprecated. Please use tf.compat.v1.image.resize_nearest_neighbor instead.

W0812 14:13:51.0803

In [21]:
callbacks = [dice_call]

In [23]:
#model.fit(X_train[0:100],Y_train[0:100], batch_size=1, epochs=10, validation_data=(X_val[0:10], Y_val[0:10]), verbose=1)

In [24]:
del model

In [26]:
del model0

In [139]:
model0 = keras.applications.resnet50.ResNet50(include_top=False, weights=None, input_tensor=None, input_shape=input_shape, pooling=None)
    #model0.layers.pop(176)
    #model0.layers.pop(175)

out12 = model0.layers[12]
out38 = model0.layers[38]
out102 = model0.layers[102]
output145 = model0.layers[145]
output174 = model0.layers[174]

activation12 = out12.output
activation38 = out38.output
activation102 = out102.output
activation145 = output145.output
activation174 = output174.output

/home/deeptek4/anaconda3/envs/deeptek3/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [90]:
final_activation_output = model0.layers[174]
final_activation = final_activation_output.output

In [80]:
final_activation

<tf.Tensor 'activation_98/Relu:0' shape=(?, 8, 8, 2048) dtype=float32>

In [91]:
activation145

<tf.Tensor 'activation_139/Relu:0' shape=(?, 8, 8, 512) dtype=float32>

In [143]:
#middle 

middle_block = Conv2D(8, (2, 2), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda))(final_activation)

In [145]:
activation12

<tf.Tensor 'activation_150/Relu:0' shape=(?, 64, 64, 64) dtype=float32>

In [146]:
activation38

<tf.Tensor 'activation_157/Relu:0' shape=(?, 64, 64, 256) dtype=float32>

In [147]:
activation102

<tf.Tensor 'activation_175/Relu:0' shape=(?, 16, 16, 1024) dtype=float32>

In [148]:
activation145

<tf.Tensor 'activation_188/Relu:0' shape=(?, 8, 8, 512) dtype=float32>

In [144]:
middle_block

<tf.Tensor 'conv2d_2/BiasAdd:0' shape=(?, 8, 8, 8) dtype=float32>

In [149]:
#final_activation  = 

#final_activation = Conv2D(64, (3, 3), padding = "same", activation='relu', kernel_regularizer=regularizers.l2(l2_lambda), name = "conv5_1")(final_activation)

In [150]:
import keras
from keras.layers.core import Reshape

In [151]:
upconv6 = Conv2DTranspose(64,(2,2), padding='same', kernel_regularizer=regularizers.l2(l2_lambda))(middle_block)
#     upconv6 = Dropout(dropout_rate)(upconv6)
#     concat6 = concatenate([conv4_2, upconv6], name = "concat6")

In [152]:
upconv6.shape

TensorShape([Dimension(None), Dimension(None), Dimension(None), Dimension(64)])

In [153]:
#upconv6 = Dropout(0.2)(upconv6)

In [154]:
upconv6

<tf.Tensor 'conv2d_transpose_20/BiasAdd:0' shape=(?, ?, ?, 64) dtype=float32>

In [155]:
 concat6 = concatenate([activation145, upconv6], name = "concat6")

In [156]:
concat6

<tf.Tensor 'concat6_12/concat:0' shape=(?, 8, 8, 576) dtype=float32>

In [163]:
conv6_1 = Conv2D(256, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv6_1")(concat6)
conv6_1 = bn(name = "conv6_1_bn")(conv6_1)
conv6_2 = Conv2D(256, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv6_2")(conv6_1)
conv6_2 = bn(name = "conv6_2_bn")(conv6_2)
conv6_reshape = Reshape((16,16,64))(conv6_2)


In [177]:
upconv7 = Conv2DTranspose(128,(3, 3), padding='same', kernel_regularizer=regularizers.l2(l2_lambda))(conv6_reshape)
upconv7 = Dropout(dropout_rate)(upconv7)
#upconv7  = keras.layers.UpSampling2D(size=(1,1))(upconv7)
concat7 = concatenate([activation102, upconv7], name = "concat7")

In [178]:
conv7_2.shape

TensorShape([Dimension(None), Dimension(16), Dimension(16), Dimension(128)])

In [179]:
conv7_1 = Conv2D(128, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv7_1")(concat7)
conv7_1 = bn(name = "conv7_1_bn")(conv7_1)
conv7_2 = Conv2D(128, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv7_2")(conv7_1)
conv7_2 = bn(name = "conv7_2_bn")(conv7_2)
conv7_reshape = Reshape((64,64,8))(conv7_2)


In [181]:
upconv8 = Conv2DTranspose(256,(3, 3), padding='same', kernel_regularizer=regularizers.l2(l2_lambda))(conv7_reshape)
upconv8 = Dropout(dropout_rate)(upconv8)

In [182]:
upconv8

<tf.Tensor 'dropout_18/cond/Merge:0' shape=(?, ?, ?, 256) dtype=float32>

In [185]:
concat8 = concatenate([activation38, upconv8], name = "concat8")
conv8_1 = Conv2D(256, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv8_1")(concat8)
conv8_1 = bn(name = "conv8_1_bn")(conv8_1)
conv8_2 = Conv2D(256, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv8_2")(conv8_1)
conv8_2 = bn(name = "conv8_2_bn")(conv8_2)

In [186]:
conv8_2

<tf.Tensor 'conv8_2_bn_2/cond/Merge:0' shape=(?, 64, 64, 256) dtype=float32>

In [187]:
activation12

<tf.Tensor 'activation_150/Relu:0' shape=(?, 64, 64, 64) dtype=float32>

In [189]:
upconv9 = Conv2DTranspose(256,(3, 3), padding='same', kernel_regularizer=regularizers.l2(l2_lambda))(conv8_2)
upconv9 = Dropout(dropout_rate)(upconv9)
concat9 = concatenate([activation12, upconv9], name = "concat9")

In [195]:
conv9_1 = Conv2D(1024, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv9_1")(concat9)
conv9_1 = bn(name = "conv9_1_bn")(conv9_1)
conv9_2 = Conv2D(1024, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv9_2")(conv9_1)
conv9_2 = bn(name = "conv9_2_bn")(conv9_2)
conv9_reshape = Reshape((256,256,64))(conv9_2)

conv10 = Conv2D(1, (1, 1), padding = "same", activation = 'sigmoid', name = "conv10")(conv9_reshape)



In [197]:
m1 = Model(model0.input,conv10)

ValueError: Graph disconnected: cannot obtain value for tensor Tensor("input_3:0", shape=(?, 256, 256, 3), dtype=float32) at layer "input_3". The following previous layers were accessed without issue: []

In [193]:
256*256

65536

In [194]:
4194304/65536

64.0

In [127]:
 concat7 = concatenate([activation102, upconv7], name = "concat6")

In [120]:
conv6_2

<tf.Tensor 'conv6_2_bn_2/cond/Merge:0' shape=(?, 8, 8, 256) dtype=float32>

In [123]:
conv6_2_reshape = Reshape((16,16,64))(conv6_2)

In [135]:
for i,layer in enumerate(model0.layers):
    print(i,'---> ', layer.name)

0 --->  input_3
1 --->  conv1_pad
2 --->  conv1
3 --->  bn_conv1
4 --->  activation_99
5 --->  pool1_pad
6 --->  max_pooling2d_3
7 --->  res2a_branch2a
8 --->  bn2a_branch2a
9 --->  activation_100
10 --->  res2a_branch2b
11 --->  bn2a_branch2b
12 --->  activation_101
13 --->  res2a_branch2c
14 --->  res2a_branch1
15 --->  bn2a_branch2c
16 --->  bn2a_branch1
17 --->  add_33
18 --->  activation_102
19 --->  res2b_branch2a
20 --->  bn2b_branch2a
21 --->  activation_103
22 --->  res2b_branch2b
23 --->  bn2b_branch2b
24 --->  activation_104
25 --->  res2b_branch2c
26 --->  bn2b_branch2c
27 --->  add_34
28 --->  activation_105
29 --->  res2c_branch2a
30 --->  bn2c_branch2a
31 --->  activation_106
32 --->  res2c_branch2b
33 --->  bn2c_branch2b
34 --->  activation_107
35 --->  res2c_branch2c
36 --->  bn2c_branch2c
37 --->  add_35
38 --->  activation_108
39 --->  res3a_branch2a
40 --->  bn3a_branch2a
41 --->  activation_109
42 --->  res3a_branch2b
43 --->  bn3a_branch2b
44 --->  activation_110


In [138]:
for i,layer in enumerate(model0.layers):
    print(i, '---->', layer.name ,'----> ', layer.output.shape)

0 ----> input_3 ---->  (?, 256, 256, 3)
1 ----> conv1_pad ---->  (?, 262, 262, 3)
2 ----> conv1 ---->  (?, 128, 128, 64)
3 ----> bn_conv1 ---->  (?, 128, 128, 64)
4 ----> activation_99 ---->  (?, 128, 128, 64)
5 ----> pool1_pad ---->  (?, 130, 130, 64)
6 ----> max_pooling2d_3 ---->  (?, 64, 64, 64)
7 ----> res2a_branch2a ---->  (?, 64, 64, 64)
8 ----> bn2a_branch2a ---->  (?, 64, 64, 64)
9 ----> activation_100 ---->  (?, 64, 64, 64)
10 ----> res2a_branch2b ---->  (?, 64, 64, 64)
11 ----> bn2a_branch2b ---->  (?, 64, 64, 64)
12 ----> activation_101 ---->  (?, 64, 64, 64)
13 ----> res2a_branch2c ---->  (?, 64, 64, 256)
14 ----> res2a_branch1 ---->  (?, 64, 64, 256)
15 ----> bn2a_branch2c ---->  (?, 64, 64, 256)
16 ----> bn2a_branch1 ---->  (?, 64, 64, 256)
17 ----> add_33 ---->  (?, 64, 64, 256)
18 ----> activation_102 ---->  (?, 64, 64, 256)
19 ----> res2b_branch2a ---->  (?, 64, 64, 64)
20 ----> bn2b_branch2a ---->  (?, 64, 64, 64)
21 ----> activation_103 ---->  (?, 64, 64, 64)
22 ----

In [ ]:
model.summary()